# Milestone 1: Frame your analysis and EDA




## 1. Project Topics

### Exploratory 1

#### Business Goals

Determine if NSFW posts post affects user interaction.

#### Technical Proposals

Check the means of the distribution of comments for each type of post in a box plot. Perform hypothesis tests. Perform hypothesis tests for statistical significance.




### Exploratory 2

#### Business Goals

Determine what is the correlation that exists between the number of comments and the score of a post.

#### Technical Proposals

Calculate correlations between the score in various selected subreddits and the number of comments in each. Perform hypothesis tests for statistical significance.


### Exploratory 3

#### Business Goals

Determine the times of the day when posts typically receive the most engagement.

#### Technical Proposals

Plot comments over time.

## 2. EDA

### Bucket checks

In [2]:
!aws s3 ls


2023-08-29 23:43:16 sagemaker-studio-692960231031-wo7kgoszj2g
2023-08-29 23:50:01 sagemaker-us-east-1-692960231031
2023-08-30 00:34:21 vad49
2023-09-16 16:02:10 vad49-labdata


In [3]:
#!aws s3 ls s3://vad49/project_lowercase_test/
!aws s3 ls s3://project17-bucket-alex/project_jan2021/

#!aws s3 cp s3://project17-bucket-alex/eda_ideas.txt -

                           PRE comments/
                           PRE submissions/


### Setup

In [ ]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.2.0 s3fs pyarrow

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")


In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import col, length, isnan, when, count

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 50) 
#pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)


In [ ]:
# Import pyspark and build Spark session

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

### Bring in submissions and comments data

In [ ]:
%%time
s3_path_submissions = f"s3a://project17-bucket-alex/project_jan2021//submissions"
print(f"reading submissions from {s3_path_submissions}")

submissions = spark.read.parquet(s3_path_submissions, header=True)


In [ ]:
%%time
s3_path_comments = f"s3a://project17-bucket-alex/project_jan2021//comments"
print(f"reading submissions from {s3_path_comments}")

comments = spark.read.parquet(s3_path_comments, header=True)


In [ ]:
submissions_small = submissions.sample(withReplacement=False, fraction=0.01, seed=42)
comments_small = comments.sample(withReplacement=False, fraction=0.01, seed=42)


In [ ]:
# create small dfs

use_small = True  # to easily swap between the small and small dfs
submissions_active = submissions_small if use_small else submissions
comments_active = comments_small if use_small else comments


In [ ]:
#cache
submissions_active.cache()
comments_active.cache()


### 2.1 Report on the basic info about your dataset. What are the interesting columns? What is the schema? How many rows do you have? etc. etc.


In [ ]:

print(f"submissions shape: ({submissions_active.count()}, {len(submissions_active.columns)})")
print(f"submissions shape: ({comments_active.count()}, {len(comments_active.columns)})")




Submissions

In [ ]:
submissions_active.printSchema()

Comments

In [ ]:
comments_active.printSchema()

### 2.2 Conduct basic data quality checks! Make sure there are no missing values, check the length of the comments, and remove rows of data that might be corrupted. Even if you think all your data is perfect, you still need to demonstrate that with your analysis.



In [ ]:

def check_and_remove_missing(df: DataFrame, threshold: int = 100) -> DataFrame:

    # Check for missing values
    missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

    # Show the missing values count for each column
    missing_values_collected = missing_values.collect()[0].asDict()
    print("Missing values in each column:")
    for column, missing_count in missing_values_collected.items():
        print(f"{column}: {missing_count}")

    # Identify columns with missing values above threshold
    columns_to_drop = [column for column, missing_count in missing_values_collected.items() if missing_count > threshold]

    # Drop the identified columns from the dataframe
    df = df.drop(*columns_to_drop)
    
    # Recalculate missing values for the updated DataFrame
    missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
    missing_values_collected = missing_values.collect()[0].asDict()
    
    # Print updated missing values count
    print("Missing values after column removal:")
    for column, missing_count in missing_values_collected.items():
        print(f"{column}: {missing_count}")
    
    return df



In [ ]:
submissions_active = check_and_remove_missing(submissions_active)



In [ ]:
# turn to int for displaying

columns_to_cast_submissions = [
    "archived", "contest_mode", "hidden", "hide_score", 
    "is_crosspostable", "is_reddit_media_domain", "is_self",
    "is_video", "locked", "over_18", "pinned", "spoiler", "stickied"
]

for column_name in columns_to_cast_submissions:
    submissions_active = submissions_active.withColumn(column_name, col(column_name).cast("integer"))

    

Let's remove submissions without a body should obviously go, but what about the submissions without a self text (deleted, removed or empty). We can keep where the author is empty.

In [ ]:

def clean_submissions(df: DataFrame) -> DataFrame:
    
    # Define a list of conditions that would indicate a row needs to be removed
    conditions = (col('selftext') != "[removed]") & (col('selftext') != "[deleted]") & (col('selftext').isNotNull() & (col('selftext') != ""))

    # Apply the filter
    cleaned_df = df.filter(conditions)
    
    return cleaned_df




In [ ]:
submissions_active = clean_submissions(submissions_active)


In [ ]:
display(submissions_active.limit(5).toPandas())

Comments

In [ ]:
comments_active = check_and_remove_missing(comments_active)



Now on to the comments.

Let's do the same for the body of the comments.

In [ ]:
def clean_comments(df: DataFrame) -> DataFrame:
    
    # Define the filter conditions
    conditions = (col('body') != "[removed]") & (col('body') != "[deleted]") & (col('body').isNotNull() & (col('body') != ""))

    # Apply the filter
    cleaned_df = df.filter(conditions)
    
    return cleaned_df




In [ ]:
comments_active = clean_comments(comments_active)


In [ ]:
# turn to ints for displaying
columns_to_cast_comments = [
    "can_gild", "stickied", "is_submitter"
]

for column_name in columns_to_cast_comments:
    comments_active = comments_active.withColumn(column_name, col(column_name).cast("integer"))





In [ ]:
display(comments_active.limit(5).toPandas())

Final shape

In [ ]:

print(f"submissions shape: ({submissions_active.count()}, {len(submissions_active.columns)})")
print(f"submissions shape: ({comments_active.count()}, {len(comments_active.columns)})")


In [ ]:
submissions_active.groupby('subreddit').count().show()



### 2.5 Use data transformations to make AT LEAST 3 new variables that are relevant to your business questions. We cannot be more specific because this depends on your project and what you want to explore!



In [ ]:

# Define the weights for num_comments and score
weight_comments = 0.5
weight_score = 0.5

# Add a new column with the weighted average of num_comments and score
submissions_active = submissions_active.withColumn(
    'comments_and_score',
    (col('num_comments') * weight_comments) + (col('score') * weight_score)
)



In [ ]:
submissions_active.printSchema()

In [ ]:

# Extract the week of year and hour from `created_utc`
comments_active = comments_active.withColumn('week_of_year', F.weekofyear('created_utc'))
comments_active = comments_active.withColumn('hour_of_day', F.hour('created_utc'))


comments_active.printSchema()

### 2.3 Produce at least 5 interesting graphs about your dataset. Think about the dimensions that are interesting for your Reddit data! There are millions of choices. Make sure your graphs are connected to your business questions.



Topic 1

In [ ]:
submissions_active.groupby('over_18').count().show()


In [ ]:


# Get the number of true cases for over_18
true_count = submissions_active.filter(col('over_18') == 1).count()

# Sample the same number of false cases
false_count = submissions_active.filter(col('over_18') == 0).count()
fraction = true_count / false_count

# Use sampleBy if you need stratified sampling to maintain a proportion
sampled_false = submissions_active.filter(col('over_18') == 0).sample(withReplacement=False, fraction=fraction, seed=42)

# Since you need exactly `true_count` number of samples, we need to take action in the sampled DataFrame
# This may cause Spark to scan the DataFrame twice
sampled_false = sampled_false.limit(true_count)

# Combine the true cases and sampled false cases
combined = submissions_active.filter(col('over_18') == 1).unionAll(sampled_false)



In [ ]:
# Now convert this combined Spark DataFrame to Pandas DataFrame for visualization (assuming the data is small enough to fit into memory)
pandas_df = combined.toPandas()

# Ensure 'over_18' is a string to be treated as categorical data
pandas_df['over_18'] = pandas_df['over_18'].map({0: 'False', 1: 'True'})


# Calculate the mean and standard deviation of num_comments
#mean_comments = pandas_df['num_comments'].mean()
#std_comments = pandas_df['num_comments'].std()

# Define the upper bound as 3 standard deviations above the mean
#upper_bound = mean_comments + 3 * std_comments


# remove rows with more than 200 comments
pandas_df = pandas_df[pandas_df['num_comments'] <= 200]

pandas_df.to_csv('../../data/eda-plots/box-plot-nsfw-data.csv', index=False)


# Filter the DataFrame to exclude any num_comments above the upper bound
#pandas_df = pandas_df[pandas_df['num_comments'] <= upper_bound]



In [ ]:
# Create a box plot for the score by over_18 status
plt.figure(figsize=(10, 6))
ax = pandas_df.boxplot(by='over_18', column=['num_comments'], grid=False)

# Set the title and labels
ax.set_title('Box Plot of Scores by Over 18 Status')
ax.set_xlabel('Over 18 Status')
ax.set_ylabel('Comments')
plt.suptitle('')  # Suppress the automatic Pandas-generated title

# Show the plot
plt.show()


In [ ]:
counts = pandas_df['over_18'].value_counts()
print("Counts for 'over_18' in the DataFrame:")
print(counts)


Topic 2

In [ ]:

# Group by 'subreddit' and calculate the mean of 'num_comments' and 'score'
subreddit_means = submissions_active.groupBy('subreddit') \
                                    .agg(F.mean('num_comments').alias('mean_num_comments'), 
                                         F.mean('score').alias('mean_score'))

# Now convert this Spark DataFrame to a Pandas DataFrame
pandas_df_means = subreddit_means.toPandas()

# Ensure the 'subreddit' is treated as a category for better plotting
pandas_df_means['subreddit'] = pandas_df_means['subreddit'].astype('category')

pandas_df_means.to_csv('../../data/eda-plots/mean-comments-vs-mean-score-data.csv', index=False)




In [ ]:

# Create the scatter plot
plt.figure(figsize=(12, 8))
scatter_plot = sns.scatterplot(data=pandas_df_means, 
                               x='mean_num_comments', 
                               y='mean_score', 
                               hue='subreddit')

# Enhance the plot
scatter_plot.set_title('Scatter Plot of Mean Comments vs. Mean Score, Hued by Subreddit')
scatter_plot.set_xlabel('Mean Number of Comments')
scatter_plot.set_ylabel('Mean Score')
plt.legend(title='Subreddit', loc='upper right')


# Show the plot
plt.show()



Topic 3

In [ ]:
from pyspark.sql.functions import weekofyear, hour, avg

from pyspark.sql.functions import col, to_date, unix_timestamp, lit

# Assuming 'created_utc' is in Unix timestamp format, convert it to timestamp type
#comments_active = comments_active.withColumn('created_utc', to_timestamp('created_utc'))



# Filter out dates on or before January 3rd, 2021
comments_active = comments_active.filter(to_date(col('created_utc')) > lit('2021-01-03'))


# Group by week of year and hour of day, then calculate the average number of comments
comments_grouped = comments_active.groupBy('week_of_year', 'hour_of_day').count()

# Pivot the data to create a matrix of week_of_year (rows) by hour_of_day (columns) with the counts of comments
comments_pivot = comments_grouped.groupBy('week_of_year').pivot('hour_of_day').avg('count').orderBy('week_of_year')

# Convert to Pandas DataFrame for visualization
comments_pivot_df = comments_pivot.toPandas().set_index('week_of_year')

comments_pivot_df.to_csv('../../data/eda-plots/average-comments-hour-and-week-data.csv', index=True)

In [ ]:
# Plotting the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(comments_pivot_df, cmap='viridis')
plt.title('Average Number of Comments per Hour and Week of the Year')
plt.xlabel('Hour of Day')
plt.ylabel('Week of the Year')
plt.show()



### 2.4 Produce at least 3 interesting summary tables about your dataset. You can decide how to split up your data into categories, time slices, etc. There are infinite ways you can make summary statistics. Be unique, creative, and interesting!



In [ ]:


from pyspark.sql.functions import col, corr

# This will calculate the correlation between 'num_comments' and 'score' for each subreddit within the original data.
correlation_by_subreddit = submissions_active.groupBy('subreddit') \
                                             .agg(corr(col('num_comments'), col('score')).alias('correlation_coefficient'))

# Now collect and show the data
correlation_by_subreddit.show()



In [ ]:

# Convert to Pandas DataFrame
pandas_df_correlation_table = correlation_by_subreddit.toPandas()

# Export to CSV
pandas_df_correlation_table.to_csv('../../data/eda-plots/correlation_by_subreddit.csv', index=False)



### 2.6 Implement regex searches for specific keywords of interest to produce dummy variables and then make statistics that are related to your business questions. Note, that you DO NOT have to do textual cleaning of the data at this point. The next assignment on NLP will focus on the textual cleaning and analysis aspect.



In [ ]:
from pyspark.sql.functions import col, regexp_extract

# Define the keywords
keywords = ['fascinating', 'entertaining', 'boring']


# Initialize the DataFrame with the original data
comments_active_dummies = comments_active

# Create dummy variables for each keyword
for keyword in keywords:
    comments_active_dummies = comments_active_dummies.withColumn(
        keyword, 
        (regexp_extract(col('body'), f'\\b{keyword}\\b', 0) != '').cast('integer')
    )

# This will hold all the pandas dataframes with the counts
combined_counts = {}

# Go through the keywords, convert the counts to pandas dataframes and collect them in a dictionary
for keyword in keywords:
    count_df = comments_active_dummies.groupBy(keyword).count().toPandas()
    count_df.set_index(keyword, inplace=True)  # Set the keyword column as the index
    combined_counts[keyword] = count_df

# Now, assuming that the index contains the same values for all dataframes, concatenate them
# The 'axis=1' parameter is used to concatenate columns, not rows
combined_df = pd.concat(combined_counts.values(), axis=1, keys=combined_counts.keys())

# Display the combined dataframe
print(combined_df)

combined_df.to_csv('../../data/eda-plots/dummies_keywords_count.csv')



### 2.7 Find some type of external data to join onto your Reddit data. Don’t know what to pick? Consider a time-related dataset. Stock prices, game details over time, active users on a platform, sports scores, covid cases, etc., etc. While you may not need to join this external data with your entire dataset, you must have at least one analysis that connects to external data. You do not have to join the external data and analyze it yet, just find it.



### If you are planning to make any custom datasets that are derived from your Reddit data, make them now. These datasets might be graph-focused, or maybe they are time series focused, it is completely up to you!